In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

num_states = 15
num_actions = 4

prob = 0.02
gamma = 0.95
theta = 0.05

a_desc = ["UP", "DOWN", "LEFT", "RIGHT"]
# 0 - UP
# 1 - DOWN
# 2 - LEFT
# 3 - RIGHT

maze = [['W','W','W','W','W','W'],
        ['W','S','S','S','S','W'],
        ['W','S','W','B','S','W'],
        ['W','S','W','S','B','W'],
        ['W','S','W','W','G','W'],
        ['W','B','S','S','W','W'],
        ['W','W','W','W','W','W']]

s_loc = np.zeros((num_states,2))

# Initialize the reward function
r = np.zeros((num_actions,num_states, num_states))

# Initialize the transition probability function
p = np.zeros((num_actions,num_states, num_states))

# Initialize the probability of taking each action in each state
# pi = np.ones((num_states, num_actions, num_states))

maze_col = len(maze[0])
maze_row = len(maze)


# Get the reward for the next state (s_prime)
def getReward(s_prime):
    reward = 0
    
    # The location of the current state in the maze
    i = s_loc[s_prime,0]
    j = s_loc[s_prime,1]
    
    if(maze[i][j] == 'B'):
        reward += -10
    elif(maze[i][j] == 'G'):
        reward += 200
    
    reward += -1
    
    return  reward

# Map between the state and the location in the maze
def mapStateToMaze():
    s = 0
    for i in range(maze_row):
        for j in range(maze_col):
            if(maze[i][j] == 'S' or maze[i][j] == 'B' or maze[i][j] == 'G'):
                s_loc[s] = [i,j]
                s += 1

mapStateToMaze()
s_loc = s_loc.astype(int)

print("The state location in the maze are: ")
print(s_loc)

# Get state from the location in the maze
def getStateFromMaze(i,j):
    k = np.where((s_loc == (i,j)).all(axis=1))
    if(k[0].size == 0):
        return 0
    else:
        return k[0][0]
            
# Get the next state given the current state and the action
def getNextState(s, a):
    s_prime = 0
    
    # The location of the current state in the maze
    i = s_loc[s,0]
    j = s_loc[s,1]
    
    if(a == 0):
        # Action is UP
        if(maze[i-1][j] != 'W'):
            s_prime = getStateFromMaze(i-1,j)
        else:
            s_prime = s
    elif(a == 1):
        # Action is DOWN
        if(maze[i+1][j] != 'W'):
            s_prime = getStateFromMaze(i+1,j)
        else:
            s_prime = s
    elif(a == 2):
        # Action is LEFT
        if(maze[i][j-1] != 'W'):
            s_prime = getStateFromMaze(i,j-1)
        else:
            s_prime = s
    elif(a == 3):
        # Action is RIGHT
        if(maze[i][j+1] != 'W'):
            s_prime = getStateFromMaze(i,j+1)
        else:
            s_prime = s
    
    # Insert the reward of the going from s to s_prime by taking action a
    if(s != 11):
        r[a][s][s_prime] = getReward(s_prime)
    else:
        r[a][s][s_prime] = 0
        
    return s_prime

getAdjacentStates = lambda s: [getNextState(s,0), getNextState(s,1), getNextState(s,2), getNextState(s,3)]

rewardMatrix = lambda s: [getReward(s_prime) for s_prime in getAdjacentStates(s)]

# Calculate the probability of the other states given the current state and action
def getOtherState(s,a):
    for i in range(num_actions):
        s_prime = getNextState(s,i)
        if(i != a):
            p[a][s][s_prime] += prob/3
        # else:
        #     p[a][s][s_prime] = prob/3


# Create the probability transition matrix
def createTransitionMatrix(prob):
    for a in range(num_actions):
        for s in range(num_states):
            next_state = getNextState(s,a)
            adjStates = getAdjacentStates(s)

            for s_prime in range(len(adjStates)):
                if(adjStates[s_prime] == next_state):
                    p[a][s][adjStates[s_prime]] = 1-prob
                    getOtherState(s,a)
                # else:
                #     p[s_prime][s][adjStates[s_prime]] = prob/3



def createRewardMatrix():
    for a in range(num_actions):
        for s in range(num_states):
            adjStates = getAdjacentStates(s)
            for s_prime in adjStates:
                r[a][s][s_prime] = rewardMatrix(s)[a]

createTransitionMatrix(prob)
# print("")




The state location in the maze are: 
[[1 1]
 [1 2]
 [1 3]
 [1 4]
 [2 1]
 [2 3]
 [2 4]
 [3 1]
 [3 3]
 [3 4]
 [4 1]
 [4 4]
 [5 1]
 [5 2]
 [5 3]]


# Problem 1

## Policy Evaluation

Value-State Function:

$
V^{\pi}(S) = \sum_{S'} P(S'|S,\pi(S))[R(S,\pi(S),S')+\gamma V^{\pi}(S')]
$

In [6]:
prob = 0
gamma = 0.95
theta = 0.5
    
# Initialize the transition probability function
p = np.zeros((num_actions,num_states, num_states))

createTransitionMatrix(prob)
createRewardMatrix()

pi_0 = np.zeros((num_states)).astype(int)
pi_0 += 2


def policy_iteration(p, r, gamma, theta, pi_0):
    pi = pi_0
    policy_stable = False
    while(policy_stable == False):
        v, delta = policy_evaluation(p,r,gamma,theta,pi)
        pi, policy_stable = policy_improvement(p,r,gamma,v,pi)
    return v, pi

def policy_evaluation(p, r, gamma, theta, initial_pi):
    v_0 = np.zeros((num_states))
    v_1 = np.zeros((num_states))

    while True:
        delta = 0
        v_0 = np.copy(v_1)
        v_1 = np.zeros((num_states))

        for s in range(num_states):
            # v_0[s] = v_1[s]
            a = initial_pi[s]
            
            # v_1[s] = 0
            
            for s_prime in range(num_states):
                if(s != 11 or s_prime != 11):
                    if(p[a][s][s_prime] != 0):
                        v_1[s] += (p[a][s][s_prime] * (r[a][s][s_prime] + (gamma * v_0[s_prime])))
                    else:
                        v_1[s] += 0
            
            v_1[11] =0
            

        for s in range(num_states):
            delta = max(delta, abs(v_0[s] - v_1[s]))
                
            if(delta < theta):
                return v_1,delta
        # print(v_1)


# Generate method for policy improvement in reinforcement learning
def policy_improvement(p, r, gamma, v, pi):
    policy_stable = True
    for s in range(num_states):
        old_action = pi[s]
        q = np.zeros((num_actions))
        
        for a in range(num_actions):
            for s_prime in range(num_states):
                if(p[a][s][s_prime] != 0):
                    q[a] += (p[a][s][s_prime] * (r[a][s][s_prime] + (gamma * v[s_prime])))
                else:
                    q[a] += 0
        
        v_index = np.argwhere(q == np.max(q)).flatten().tolist()
        if(len(v_index) > 1):
            for m in range(len(v_index)):
                next_state = getNextState(s,v_index[m])

                # The location of the current state in the maze
                i = s_loc[next_state,0]
                j = s_loc[next_state,1]

                if(v_index[m] == 0):
                    # Action is UP
                    if(maze[i-1][j] == 'W'):
                        v_index.pop(m)
                elif(v_index[m] == 1):
                    # Action is DOWN
                    if(maze[i+1][j] == 'W'):
                        v_index.pop(m)
                elif(v_index[m] == 2):
                    # Action is LEFT
                    if(maze[i][j-1] == 'W'):
                        v_index.pop(m)
                elif(v_index[m] == 3):
                    # Action is RIGHT
                    if(maze[i][j+1] == 'W'):
                        v_index.pop(m)
        else:
            pi[s] = np.argmax(q)
        
        if(old_action != pi[s]):
            policy_stable = False
            
    print(v)
    print(pi)
    return pi, policy_stable

v, pi = policy_iteration(p, r, gamma, theta, pi_0)

# print("The value function is: ")
# print(v)
# print("The optimal policy is: ")
# print(pi)

IndexError: list index out of range

## Value Iteration Backup

$
V_{k+1}(S) = max_{a \in A} \sum_{S'} \; P(S'|S,a) \; [R(S,a,S')+ \gamma \; V_{k}(S')]
$

In [102]:
prob = 0
gamma = 1
theta = 0.5

# Initialize the transition probability function
p = np.zeros((num_actions,num_states, num_states))
v_result = np.zeros((num_states))

createTransitionMatrix(prob)
createRewardMatrix()

def vib(p, r, gamma, theta):
    v_0 = np.zeros((num_actions, num_states))
    v_1 = np.zeros((num_actions, num_states))

    while True:
        delta = 0
        v_0 = np.copy(v_1)
        v_1 = np.zeros((num_actions, num_states))

        for s in range(num_states):
            for a in range(num_actions):
        
                # v_1[a][s] = 0

                for s_prime in range(num_states):
                    if(s != 11 or s_prime != 11):
                        if(p[a][s][s_prime] != 0):
                            v_1[a][s] += (p[a][s][s_prime] * (r[a][s][s_prime] + (gamma * v_result[s_prime])))
                        else:
                            v_1[a][s] += 0
        
            # Find the maximum value state for all actions
            v_result[s] = np.max(v_1[:,s])
            
            # Always set the value of the goal state to 0
            v_result[11] = 0

            for i in range(num_actions):
                delta = max(delta, abs(v_0[i][s] - v_1[i][s]))
            
            if(delta < theta):
                return v_result,v_0,delta
        # print(v_0)
        print(v_result)


v_result,q,delta = vib(p,r,gamma,theta)
# print(v_result)

for i in range(num_states):
    print(i,np.argmax(q[:,i]))
    # print(np.max(q[:,i]))

[ -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1. 199.  -1.   0.  -1.  -1.
  -1.]
[ -2.  -2.  -2.  -2.  -2.  -2. 188.  -2. 188. 199.  -2.   0.  -2.  -2.
  -2.]
[ -3.  -3.  -3. 187.  -3. 187. 188.  -3. 188. 199.  -3.   0.  -3.  -3.
  -3.]
[ -4.  -4. 186. 187.  -4. 187. 188.  -4. 188. 199.  -4.   0.  -4.  -4.
  -4.]
[ -5. 185. 186. 187.  -5. 187. 188.  -5. 188. 199.  -5.   0.  -5.  -5.
  -5.]
[184. 185. 186. 187. 183. 187. 188. 182. 188. 199. 181.   0. 180. 169.
 168.]
[184. 185. 186. 187. 183. 187. 188. 182. 188. 199. 181.   0. 180. 169.
 168.]
0 3
1 3
2 3
3 1
4 0
5 1
6 1
7 0
8 3
9 1
10 0
11 0
12 0
13 2
14 2


In [ ]:
maze = [['W','W','W','W','W','W'],
        ['W','S','S','S','S','W'],
        ['W','S','W','B','S','W'],
        ['W','S','W','S','B','W'],
        ['W','S','W','W','G','W'],
        ['W','B','S','S','W','W'],
        ['W','W','W','W','W','W']]

# This is a random matrix for example purposes. 
# Matrix is defined as 20x20 instead of 18x18 stated in the project description in order to treat borders as wall states
State_Matrix = \
    np.array([[9,9,9,9,9,9],
              [9,0,0,0,0,9],
              [9,0,9,1,0,9],
              [9,0,9,0,1,9],
              [9,0,9,9,7,9],
              [9,1,0,0,9,9],
              [9,9,9,9,9,9]])
        


plt.subplots(figsize=(10,7.5))
heatmap = sns.heatmap(State_Matrix, fmt='', linewidths=0.25, linecolor='black', cbar= False, cmap= 'rocket_r', vmax=9, vmin=0)
heatmap.set_facecolor('black') # Color for the NaN cells in the state matrix
plt.title('Maze Problem')
plt.show()

In [ ]:
""" Function to always color the oil, bump, start, and green blocks.
 States are in the form of a list of (i,j) coordinates on the state matrix"""
def coloring_blocks(heatmap, oil_states, bump_states, start_state, end_state):
    # Adding red oil blocks
    for i in range(len(oil_states)):
        heatmap.add_patch(Rectangle((oil_states[i][1], oil_states[i][0]), 1, 1,
                                    fill=True, facecolor='red', edgecolor='red', lw=0.25))
    # Adding salmon bump blocks
    for i in range(len(bump_states)):
        heatmap.add_patch(Rectangle((bump_states[i][1], bump_states[i][0]), 1, 1,
                                    fill=True, facecolor='lightsalmon', edgecolor='lightsalmon', lw=0.25))
    # Adding start block (Blue)
    heatmap.add_patch(Rectangle((start_state[1], start_state[0]), 1, 1,
                                fill=True, facecolor='lightblue', edgecolor='lightblue', lw=0.25))

    # Adding end block (Green)
    heatmap.add_patch(Rectangle((end_state[1], end_state[0]), 1, 1,
                                fill=True, facecolor='lightgreen', edgecolor='lightgreen', lw=0.25))

# Define heatmap first
plt.subplots(figsize=(10, 7.5))
heatmap = sns.heatmap(State_Matrix, fmt=".2f", linewidths=0.25, linecolor='black', cbar=False, cmap='rocket_r')
heatmap.set_facecolor('black') 
# coloring_blocks(heatmap, oil_states=[(3,5),(4,6)], bump_states=[(13,15),(14,16)], \
#                 start_state=(2,2),end_state=(12,12))
    

# Plot the route from the start state to the end state.
# This is just an example, you may want to keep pi* coordinates and actions in a different way
path = [((1,1),'right'), ((3,4),'down'), ((4,4),'right'), ((4,5),'down'), \
        ((5,5),'right'), ((5,6),'down'), ((6,6),'right'), ((6,7),'down')]
for state_cr, direction in path:
    r = state_cr[0] # x_coordinate
    c = state_cr[1] # y_coordinate

    if direction == 'right':
        plt.arrow(c + 0.5, r + 0.5, 0.8, 0, width=0.04, color='blue')   # Right
    if direction == 'left':
        plt.arrow(c + 0.5, r + 0.5, -0.8, 0, width=0.04, color='black')  # Left
    if direction == 'up':
        plt.arrow(c + 0.5, r + 0.5, 0, -0.8, width=0.04, color='black')  # Up
    if direction == 'down':
        plt.arrow(c + 0.5, r + 0.5, 0, 0.8, width=0.04, color='black')  # Down

# Show plot
plt.show()